In [3]:
# 自动加载修改过的文件
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
sys.path.append('../')  # 返回notebook的上一级目录
warnings.filterwarnings("ignore")

In [18]:
from utils.config import *
from utils.saveLoader import load_text, get_text, get_proc_text, load_dataset

In [26]:
train_seg, test_seg = load_dataset(TRAIN_SEG, TEST_SEG)
proc_text = load_text(PROC_TEXT)
raw_text = load_text(RAW_TEXT)

In [19]:
proc1 = get_proc_text(train_seg, test_seg)
proc2 = get_text(train_seg, test_seg)

程序get_proc_text()共耗时1.54秒
程序get_text()共耗时1.53秒


In [27]:
raw_text[:1000]

'方向机重，助力泵，方向机都换了还是一样 技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了方向机带的有|车主说：[图片]|技师说：[语音]|车主说：有助力就是重，这车要匹配吧|技师说：不需要|技师说：你这是更换的部件有问题|车主说：跑快了还好点，就倒车重的很。|技师说：是非常重吗|车主说：是的，累人|技师说：[语音]|车主说：我觉得也是，可是车主是以前没这么重，选吧助理泵换了不行，又把放向机换了，现在还这样就不知道咋和车主解释。|技师说：[语音]|技师说：[语音] 随时联系\n奔驰ML500排气凸轮轴调节错误 技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障灯、显示是失火、有点缺缸、现在又没有故障、发动机多少有点抖动、检查先前的故障是报这个故障|车主说：稍等|车主说：显示图片太大传不了|技师说：[语音]|车主说：这个对发动机的抖动、失火、缺缸有直接联系吗？|技师说：[语音]|车主说：还有就是报（左右排气凸轮轴作动电磁铁）对正极短路、对地短路、对导线断路|技师说：[语音]|车主说：这几个电磁阀和问您的第一个故障有直接关系吧|技师说：[语音]|车主说：这个有办法检测它好坏吗？|技师说：[语音]|车主说：谢谢|技师说：不客气 随时联系\n2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯动，行驶升降档正常，4轮离地换挡无冲击感，更换变速箱油12L无改变。试过一辆2014年进口X1原地换挡位也有冲击感，这是什么情况，哪里的问题 技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否调整到正常液位呢|车主说：从N到D进本没有 N-R有，P-R有，最主要是行驶中到红绿灯路口等红灯停车D-N有冲击感，绿灯后N-D冲击感很小|技师说：第一，要把变速箱油位调整到标准液位，第二，清除变速箱适应值，第三升级变速箱程序，还有遇到过液力变矩器问题的|车主说：升级变速箱程序是刷模块吗|车主说：还有个问题就是停车后档位在P挡松开刹车踏板时，感觉车辆会动一下|车主说：怎样清除变速箱适应值|技师说：先从简单的排查吧，有可能程序问题，也有可能液力变矩器轴头磨损，泄压了，需要专用电脑清除变速箱适应值升级变速箱程序|技师说：换变速箱油有焦糊

In [20]:
a1 = proc1.split("\n")
a2 = proc2.split("\n")
len(a1), len(a2)

(288828, 102942)

In [23]:
train_seg.shape,test_seg.shape

((82943, 6), (20000, 5))

In [24]:
82943*3 + 20000*2

288829

In [28]:
import re

In [33]:
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.



In [36]:
re.sub(r"\n","", "我\n裂开\n了")

'我裂开了'